Used this script to generate and save quarternion data

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import ahrs
%matplotlib inline

In [5]:
line = "9,1,5b,9,9,9,9,9,9,7,7b,9,4,4b,9,"
tca_inds = line.split(',')
alt_address_list = []
tca_inds = tca_inds[:-1]
for i in range(len(tca_inds)):
    if len(tca_inds[i]) == 1: # alternate
        tca_inds[i] = int(tca_inds[i])
        alt_address_list.append(False)
    elif len(tca_inds[i]) > 1:
        tca_inds[i] = int(tca_inds[i][0])
        alt_address_list.append(True)

In [6]:
sensor_inds = tca_inds[1:]
alt_address_list = alt_address_list[1:]
sensor_list = []
sensor_ind_list = []
sensor_number = []
sensor_cnt = 0
sensor_rot = []
sensor_rot_type = [0,0,1,1,3,2,2,3,1,1,1,2,2,2] # define rotation types
sensor_labels_full = ['pelvis_imu','torso_imu','femur_l_imu','tibia_l_imu','calcn_l_imu','femur_r_imu','tibia_r_imu','calcn_r_imu','humerus_l_imu','ulna_l_imu','hand_l_imu','humerus_r_imu','ulna_r_imu','hand_r_imu']
sensor_label_list = []
for i, s_ind in enumerate(sensor_inds):
    if s_ind != 9:
        sensor_ind_list.append(s_ind)
        len_sensor_list = len(sensor_ind_list)
        sensor_number.append(sensor_cnt)
        sensor_cnt += 1
        sensor_rot.append(sensor_rot_type[i]) # say for this number sensor how to rotate it
        sensor_label_list.append(sensor_labels_full[i])

In [7]:
rot_inds = sensor_rot
rot_inds

[0, 0, 1, 1, 2, 2]

In [8]:
d2g = ahrs.common.DEG2RAD
z_neg_90 = np.array([[0,1.0,0],[-1.,0,0],[0,0,1.0]])
y_180 = np.array([[-1.0,0,0],[0,1.0,0],[0,0,-1.0]])
z_180 = np.array([[-1.0,0,0],[0,-1.0,0],[0,0,1.0]])
y_neg_90 = np.array([[0,0,-1.0],[0,1.0,0],[1.0,0,0]])
y_pos_90 = np.array([[0,0,1.0],[0,1.0,0],[-1.0,0,0]])
ankle_offset = -100.*d2g 
x_pos_ankle = np.array([[1.0,0,0],[0,np.cos(ankle_offset),-np.sin(ankle_offset)],[0,np.sin(ankle_offset),np.cos(ankle_offset)]])
hip_rot = np.matmul(y_neg_90,z_180)
foot_rot = np.matmul(x_pos_ankle, hip_rot) 
r_leg_rot = z_neg_90 
l_leg_rot = np.matmul(z_neg_90,y_180)
rot_mats = np.zeros((len_sensor_list,3,3))
for i in range(len_sensor_list): # define rotation type
    if rot_inds[i] == 0: # hip, torso, head
        rot_mats[i,:,:] = hip_rot
    elif rot_inds[i] == 1: # left side
        rot_mats[i,:,:] = l_leg_rot
    elif rot_inds[i] == 2: # right side
        rot_mats[i,:,:] = r_leg_rot
    elif rot_inds[i] == 3: # foot
        rot_mats[i,:,:] = foot_rot

In [11]:
rot_mats[1].shape

(3, 3)

In [13]:
raw_imu = np.load(r"C:\Users\chris\Documents\Visualisation\Opensim\Results\23-04-03_1757\raw_imu.npy")

In [18]:
new_imu = np.zeros(raw_imu.shape)

In [21]:
t_offset = 0
for i in range(6):
    s_off = i*6
    accel = np.matmul(raw_imu[:,s_off+t_offset:s_off+t_offset+3],rot_mats[i,:,:])
    gyro = np.matmul(raw_imu[:,s_off+t_offset+3:s_off+t_offset+6],rot_mats[i,:,:])

    new_imu[:,s_off:s_off+3] = accel
    new_imu[:,s_off+3:s_off+6] = gyro

In [43]:
Quats = "0.6845986384757144,0.5992073308343492,-0.23389679834861177,0.3428812718622028	0.6748422967666571,0.7322308980155436,-0.07630167852397637,0.05102783887719911	0.6714154897949585,0.7408884714097432,-0.014156583842807305,0.009225189940144077	0.6516008630338784,0.7101102297608508,0.14260982830912902,0.22543782679962424	0.7901597551652895,0.6002397139983825,-0.1188489944255996,-0.035139202897806955	0.559938594293556,0.33001825327312895,-0.5748585299362048,-0.4970859017178973"
Quats = Quats.split("\t")
Quats = list(map(lambda x: x.split(","),Quats))

for i in Quats:
    print(i)

['0.6845986384757144', '0.5992073308343492', '-0.23389679834861177', '0.3428812718622028']
['0.6748422967666571', '0.7322308980155436', '-0.07630167852397637', '0.05102783887719911']
['0.6714154897949585', '0.7408884714097432', '-0.014156583842807305', '0.009225189940144077']
['0.6516008630338784', '0.7101102297608508', '0.14260982830912902', '0.22543782679962424']
['0.7901597551652895', '0.6002397139983825', '-0.1188489944255996', '-0.035139202897806955']
['0.559938594293556', '0.33001825327312895', '-0.5748585299362048', '-0.4970859017178973']


In [54]:
imu_data = new_imu

rows = imu_data.shape[0] + 1 #include the first quat
Q = np.tile([1., 0., 0., 0.], (rows, 6))

Q[0].shape

for i in range(6):
    Q[0,4*i:4*i + 4] = list(map(float,Quats[i]))



In [55]:
Q[0]

array([ 0.68459864,  0.59920733, -0.2338968 ,  0.34288127,  0.6748423 ,
        0.7322309 , -0.07630168,  0.05102784,  0.67141549,  0.74088847,
       -0.01415658,  0.00922519,  0.65160086,  0.71011023,  0.14260983,
        0.22543783,  0.79015976,  0.60023971, -0.11884899, -0.0351392 ,
        0.55993859,  0.33001825, -0.57485853, -0.4970859 ])

In [56]:
mahony = ahrs.filters.Mahony(frequency = float(50))

for row in range(1,rows):
    for sn in range(6):
        imu_readings = imu_data[row-1,sn*6:sn*6 + 6]
        accel_imu = imu_readings[:3]
        gyro_imu = imu_readings[3:]

        Q[row,4*(sn):4*(sn+1)] = mahony.updateIMU(
            Q[row-1,4*(sn):4*(sn+1)],
            gyr=gyro_imu,
            acc=accel_imu)



In [57]:
Q

array([[ 0.68459775,  0.59811618, -0.23480425, ...,  0.33230115,
        -0.57372687, -0.49865997],
       [ 0.68453494,  0.59717274, -0.23561242, ...,  0.33476372,
        -0.57230797, -0.50018248],
       [ 0.68444457,  0.5961433 , -0.23654677, ...,  0.33687224,
        -0.57099988, -0.50187947],
       ...,
       [ 0.63869672,  0.5525625 , -0.29418063, ...,  0.40305098,
        -0.36329151, -0.40337711],
       [ 0.63842794,  0.55290106, -0.29405503, ...,  0.40452809,
        -0.36528336, -0.40284754],
       [ 0.63842794,  0.55290106, -0.29405503, ...,  0.40452809,
        -0.36528336, -0.40284754]])

In [60]:
def quat_to_sto(Q,sto_filename,new_sto_filename):
    #Copy format of the tiny_file.sto
    f = open(sto_filename,"r")
    lines = f.readlines()

    new_file = open(new_sto_filename,"w")
    for i in range(6):
        new_file.write(lines[i])

    data_rate = float(lines[0].split("=")[-1])
    dt = 1/data_rate

    for i in range(1,Q.shape[0]): #do not write the first quat, part of calibration
        time_stamp = (i-1)*dt
        new_file.write("{}".format(round(time_stamp,2)))

        for sensor in range(6):
            start_index = sensor * 4
            new_file.write("\t{},{},{},{}".format(
                Q[i,start_index],
                Q[i,start_index+1],
                Q[i,start_index+2],
                Q[i,start_index+3]))
        new_file.write("\n")
    
    new_file.close()

sto_filename = r"C:\Users\chris\Documents\Visualisation\Opensim\Results\23-04-03_1757\tiny_file.sto"
new_sto_file = r"C:\Users\chris\Documents\Visualisation\Opensim\Results\23-04-03_1757\processed_tiny_file.sto"

In [61]:
quat_to_sto(Q,sto_filename,new_sto_file)

In [63]:
np.save(r"C:\Users\chris\Documents\Visualisation\Opensim\rot_mats.npy",rot_mats)

In [2]:
imu_data = np.random.rand(1,36)

In [3]:
imu_data

array([[0.25772837, 0.58753477, 0.87024191, 0.10421668, 0.3661047 ,
        0.9237946 , 0.11599963, 0.98932666, 0.15463592, 0.59367452,
        0.25487759, 0.08941565, 0.95451599, 0.40361785, 0.16547417,
        0.19003998, 0.90507174, 0.37142246, 0.35131373, 0.63586574,
        0.69417198, 0.03396064, 0.11623151, 0.36747278, 0.64809746,
        0.33497764, 0.65180607, 0.91655819, 0.08832065, 0.91139443,
        0.42490401, 0.80814787, 0.82843974, 0.17172148, 0.51087878,
        0.16535852]])

In [12]:
Q = np.tile([1,0,0,0],(1,6))

In [14]:
Q.shape

(1, 24)

In [15]:
np.array([[0,0,0],[1,0,1],[1,0,1]])

array([[0, 0, 0],
       [1, 0, 1],
       [1, 0, 1]])

In [16]:
Q = np.tile([0.25,0.9,0.9,0.4],(1,6))
Q=Q.reshape((6,4))
Q

array([[0.25, 0.9 , 0.9 , 0.4 , 0.25, 0.9 , 0.9 , 0.4 , 0.25, 0.9 , 0.9 ,
        0.4 , 0.25, 0.9 , 0.9 , 0.4 , 0.25, 0.9 , 0.9 , 0.4 , 0.25, 0.9 ,
        0.9 , 0.4 ]])